In [1]:
import numpy as np

In [2]:
np.ones(5) /10

array([ 0.1,  0.1,  0.1,  0.1,  0.1])

In [3]:
np.ones(5)

array([ 1.,  1.,  1.,  1.,  1.])

In [4]:
np.mean(np.array([[.3,.6,.9], [.2,.5,1]]), axis=0)

array([ 0.25,  0.55,  0.95])

In [ ]:
def distributionalOracleOneVsAll(distribution, models, x, y, alpha, targeted=False):
    candidates = []
    # we should only take into consideration models that we could feasibly trick

    num_models = len(models)

    labels_values = []
    for labels in product(range(10), repeat=num_models):  # iterate over all possible regions
        if targeted:
            is_misclassified = (np.array(labels) == y).astype(np.float32)
        else:
            is_misclassified = (np.array(labels) != y).astype(np.float32)
        value = np.dot(is_misclassified, distribution)
        labels_values.append((labels, value))

    values = sorted(set([value for label, value in labels_values]), reverse=True)

    for curr_value in values:
        feasible_candidates = []
        for labels in [labels for labels, val in labels_values if val == curr_value]:
            v = tryRegionOneVsAll(models, labels, x)
            if v is not None:
                norm = np.linalg.norm(v)
                if norm <= alpha:
                    feasible_candidates.append((v, norm))
        # amongst those with the max value, return the one with the minimum norm
        if feasible_candidates:
            # break out of the loop since we have already found the optimal answer
            return min(feasible_candidates, key=lambda x: x[1])[0]
    return np.zeros(x.shape[0])  # we can't trick anything

In [ ]:
def coordinateAscentMulti(distribution, models, x, y, alpha, greedy=False, targeted=False):
    # targeted needs to be the target label
    num_models = len(models)

    sol = np.zeros(x.shape)
  
    labels = [y] * num_models  # initialize to the original point, of length feasible_models
    label_options = range(10)
    del label_options[y]

    model_options = dict(zip(range(num_models), distribution))
    for i in xrange(num_models):
        if greedy:  # select model with the highest probability weight
            coord = max(model_options, key=model_options.get)
            if targeted:
                labels[coord] = targeted
            else:
                labels[coord] = greedy[y]
        else:  # random: select a random model
            coord = np.random.choice(model_options.keys())  
            if targeted:
                labels[coord] = targeted
            else:
                labels[coord] = np.random.choice(label_options)

        del model_options[coord]

        v = tryRegionOneVsAll(models, labels, x)
        valid_sol = False
        if v is not None:
            norm = np.linalg.norm(v)
            if norm <= alpha:
                valid_sol = True
                sol = v
        if not valid_sol:
            break
    return sol

In [21]:
data_dims, box_vals = (28, 1, 10), (-.5, .5) if True else (224, 3, 1000), (0.0, 255.0)


ValueError: too many values to unpack

In [ ]:
def gradientDescentTargeted(distribution, models, x, target, alpha, targeted=False, learning_rate=.001, T=3000, 
                            early_stop=5):
    v = np.zeros(len(x))
    best_sol = (sys.maxint, v)
    loss_queue = []
    for i in xrange(T):
        gradient = sum([-1 * p * model.gradient(np.array([x + v]), [target]) for p, model in zip(distribution, models)])[0]

        v += learning_rate * gradient
        norm = np.linalg.norm(v)
        if norm >= alpha:
            v = v / norm * alpha

        loss = np.dot(distribution, [model.rhinge_loss([x + v], [target])[0] for model in models])

        loss_queue = [loss] + loss_queue
        if i >= early_stop:
            del loss_queue[-1]
            val = loss_queue[-1]
            if sum([val == q_val for q_val in loss_queue]) == early_stop:
                break

        if loss < best_sol[0]:
            best_sol = (loss, v)

        if loss == 0:
            break
    if targeted:
        res = best_sol[1]
    else:
        res = best_sol
    return res

In [5]:
Y_exp = np.load("multiclass_data_2/Y_exp.npy")

In [9]:
Target_exp = []
for y in Y_exp:
    options = range(10)
    del options[y]
    Target_exp.append(np.random.choice(options))

In [12]:
np.save("multiclass_data_2/Target_exp.npy", Target_exp) 

In [13]:
t = np.load("multiclass_data_2/Target_exp.npy")

In [14]:
t

array([1, 9, 9, 5, 9, 7, 0, 4, 7, 6, 9, 8, 2, 5, 6, 6, 6, 5, 5, 5, 8, 5, 6,
       2, 6, 5, 2, 5, 0, 6, 4, 8, 2, 7, 9, 7, 1, 1, 7, 5, 3, 2, 4, 5, 8, 5,
       0, 1, 8, 9, 2, 8, 0, 8, 3, 1, 0, 2, 2, 8, 2, 6, 6, 5, 6, 9, 1, 0, 5,
       6, 0, 1, 3, 0, 8, 8, 8, 7, 5, 5, 5, 3, 7, 2, 1, 0, 1, 6, 1, 1, 0, 8,
       4, 9, 7, 9, 8, 5, 3, 1])

In [15]:
zip([1,2], [5,34], [6,7])

[(1, 5, 6), (2, 34, 7)]

In [19]:
if np.array([1,43]) is not False:
    print 1

1


In [20]:
if 4:
    print 2

2
